In [ ]:
import os
import json
from openai import OpenAI
from datetime import datetime
from zoneinfo import ZoneInfo
from dotenv import load_dotenv


# .env 환경변수 로드
load_dotenv()
AZURE_OAI_ENDPOINT = os.getenv("AZURE_OAI_ENDPOINT")
AZURE_OAI_KEY = os.getenv("AZURE_OAI_KEY")
AZURE_OAI_DEPLOYMENT = os.getenv("AZURE_OAI_DEPLOYMENT")

# Initialize the Azure OpenAI client
client = OpenAI(
    base_url="https://8ai022-openai.openai.azure.com/openai/v1/",
    api_key=os.getenv("AZURE_OAI_KEY")
)

# Define the deployment you want to use for your chat completions API calls

deployment_name = os.getenv("AZURE_OAI_DEPLOYMENT")

# Simplified timezone data
TIMEZONE_DATA = {
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris",
    "seoul": "Asia/Seoul"
}

# 사용자 정의 함수.
# 모델 호출 결과인, 도시 정보를 바탕으로 현재 시간을 return
def get_current_time(location):
    """Get the current time for a given location"""
    print(f"get_current_time called with location: {location}")  
    location_lower = location.lower()
    
    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            print(f"Timezone found for {key}")  
            current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })
    
    print(f"No timezone data found for {location_lower}")  
    return json.dumps({"location": location, "current_time": "unknown"})

def run_conversation():
    # Initial user message
    #messages = [{"role": "user", "content": "What's the current time in Seoul"}] # Single function call
    messages = [{"role": "user", "content": "What's the current time in San Francisco, Tokyo, and Paris?"}] # Parallel function call with a single tool/function defined

    # 요청 시, 함수 전달하기 위한 정보
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_time",                                     # 함수 명칭
                "description": "Get the current time in a given location",      # 함수 설명
                "parameters": {                                                 # 매개변수 
                    "type": "object",                                           # 매개변수 타입
                    "properties": {
                        "location": {                                           # 매개변수 location
                            "type": "string",                                   # location 타입 == string
                            "description": "The city name, e.g. San Francisco",
                        },
                    },
                    "required": ["location"],                                   # 필수
                },
            }
        }
    ]

    # 첫번 째 API : city Name 정보를 영문으로 조회함. 
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response:")  
    print(response_message)  

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "get_current_time":
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")  
                # 전달받은 city Name 기준 현재 시간정보 조회
                time_response = get_current_time(
                    location=function_args.get("location")
                )
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "get_current_time",
                    "content": time_response,
                })
    else:
        print("No tool calls were made by the model.")  

    # 두번 째 API : 도시 정보와 도시의 현재시간 정보를 자연스럽게 출력하도록 호출
    final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
    )

    return final_response.choices[0].message.content

# Run the conversation and print the result
print(run_conversation())

Model's response:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_CDAfcOOmWpYr41IZDJYD72vW', function=Function(arguments='{"location": "San Francisco"}', name='get_current_time'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_jeN7SLpSZxQQuQxUG11XP2L8', function=Function(arguments='{"location": "Tokyo"}', name='get_current_time'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_kVqtYFYiSX8fxTSxkzwaihUD', function=Function(arguments='{"location": "Paris"}', name='get_current_time'), type='function')])
Function arguments: {'location': 'San Francisco'}
get_current_time called with location: San Francisco
Timezone found for san francisco
Function arguments: {'location': 'Tokyo'}
get_current_time called with location: Tokyo
Timezone found for tokyo
Function arguments: {'location': 'Paris'}
get_current_time called with location: Par